In [61]:
import os
import tiktoken
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

In [62]:
# Load environment variables
load_dotenv(dotenv_path="config/environment_variables.env")

client = AzureOpenAI(
  azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT_TEXT"),
  api_key=os.environ.get("AZURE_OPENAI_KEY_TEXT"),
  api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
)

In [63]:
generation_prompt = "../../data/prompt/generation.txt"
model = 'gpt-35-turbo-16k'

# get generation prompt text
with open(generation_prompt, "r") as file:
    generation_prompt = file.read()
    
generation_prompt

'Anda akan menerima konsideran dari peraturan perundang-undangan (PUU) yang berjudul "{doc_title}". Tugas anda adalah mengusulkan topik-topik umum (general) dari konsideran PUU yang diberikan.\n\n[Instruksi]\nUsulkan topik-topik yang dibahas dalam konsideran PUU yang diberikan.\n1. Label topik harus dapat digeneralisasikan seluas mungkin. Label tersebut tidak boleh terlalu spesifik atau terfokus pada PUU yang diberikan.\n2. Label topik yang diusulkan harus berupa kata benda yang mewakili subjek atau objek tunggal, maksimal tiga kata.\n3. Berikan respon dalam format "Label 1; Label 2; Label 3; dst.".\n\n[Konsideran]\nMenimbang: {doc_consideration}\n\nMohon hanya berikan label topik yang diusulkan dalam respon Anda.\n[Respon Anda]'

In [64]:
def api_call(model, prompt, max_tokens, temperature, top_p,):
    response = client.chat.completions.create(
            model=model,
            temperature=float(temperature),
            max_tokens=int(max_tokens),
            top_p=float(top_p),
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
        )
    return response.choices[0].message.content


def count_token(messages, model="gpt-35-turbo-16k"):
    if model.startswith("gpt"):
        try:
            encoding = tiktoken.encoding_for_model(model)
        except KeyError:
            encoding = tiktoken.get_encoding("cl100k_base")

        num_tokens = 0
        num_tokens += len(encoding.encode(messages))
        num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
        return num_tokens
    else:  # Open-source models
        return len(tiktoken.get_encoding("cl100k_base").encode(messages))

In [65]:
docs_df = pd.read_csv("../../data/law_analyzer_new99_keywords.csv")
docs_df

,nama_file,nomor_peraturan,judul,tahun,nomor,konsideran,heading,definition_term
0,01~pmk.010~2011.json,PMK 01/PMK.010/2011,Perubahan atas Peraturan Menteri Keuangan Nomo...,2011,1,a. bahwa dalam rangka memperoleh perhitungan ...,[],NaN
1,02~pmk.05~2011.json,PMK 02/PMK.05/2011,Pedoman Akuntansi Dan Pelaporan Aset Berupa Ba...,2011,2,a. bahwa berdasarkan Peraturan Menteri Keuang...,"['Pengertian', 'Ruang Lingkup', 'Sistem Akunta...","['Kontraktor Kontrak Kerja Sama,', 'Barang yan..."
2,02~pmk.07~2012.json,PMK 02/PMK.07/2012,Perkiraan Alokasi Dana Bagi Hasil Sumber Daya ...,2012,2,bahwa untuk melaksanakan ketentuan Pasal 27 a...,[],['Alokasi Dana Bagi Hasil Sumber Daya Alam Keh...
3,03~pmk.011~2012.json,PMK 03/PMK.011/2012,Tata Cara Pengelolaan Dan Pertanggungjawaban F...,2012,3,a. bahwa dalam rangka mendukung usaha pemanfa...,"['Tujuan', 'Prinsip', 'Bentuk', 'Ruang Lingkup...",['Badan Usaha Penyedia Jasa Penyiapan Proyek K...
4,04~pmk.02~2014.json,PMK 04/PMK.02/2014,Perubahan Kedua atas Peraturan Menteri Keuanga...,2014,4,a. bahwa dalam rangka mendukung agar kondisi ...,[],NaN
...,...,...,...,...,...,...,...,...
1631,uu-9-tahun-1992.json,UU 9 TAHUN 1992,Keimigrasian,1992,9,a. bahwa pengaturan keimigrasian yang meliput...,[],"['Keimigrasian', 'Wilayah Negara Republik Indo..."
1632,uu-9-tahun-1995.json,UU 9 TAHUN 1995,Usaha Kecil,1995,9,a. bahwa negara Republik Indonesia yang berda...,[],"['Usaha Kecil', 'Usaha Menengah dan Usaha Besa..."
1633,uu-9-tahun-2009.json,UU 9 TAHUN 2009,Badan Hukum Pendidikan,2009,9,a. bahwa untuk mewujudkan fungsi dan tujuan p...,[],"['Badan hukum pendidikan', 'Badan Hukum Pendid..."
1634,uu-9-tahun-2015.json,UU 9 TAHUN 2015,Perubahan Kedua atas Undang-Undang Nomor 23 Ta...,2015,9,a. bahwa untuk kesinambungan kepemimpinan di ...,[],NaN


In [66]:
# drop rows with missing values in konsideran
#docs_df = docs_df.dropna(subset=["konsideran"])

In [67]:
# count token for each konsideran
docs_df["token_count"] = docs_df["konsideran"].apply(count_token)
# filter rows with token >= 202 and <= 420
docs_df = docs_df[(docs_df["token_count"] >= 202) & (docs_df["token_count"] <= 420)]

In [68]:
docs_df = docs_df[:5]
docs_df

,nama_file,nomor_peraturan,judul,tahun,nomor,konsideran,heading,definition_term,token_count
0,01~pmk.010~2011.json,PMK 01/PMK.010/2011,Perubahan atas Peraturan Menteri Keuangan Nomo...,2011,1,a. bahwa dalam rangka memperoleh perhitungan ...,[],NaN,258
1,02~pmk.05~2011.json,PMK 02/PMK.05/2011,Pedoman Akuntansi Dan Pelaporan Aset Berupa Ba...,2011,2,a. bahwa berdasarkan Peraturan Menteri Keuang...,"['Pengertian', 'Ruang Lingkup', 'Sistem Akunta...","['Kontraktor Kontrak Kerja Sama,', 'Barang yan...",397
4,04~pmk.02~2014.json,PMK 04/PMK.02/2014,Perubahan Kedua atas Peraturan Menteri Keuanga...,2014,4,a. bahwa dalam rangka mendukung agar kondisi ...,[],NaN,341
5,04~pmk.06~2010.json,PMK 04/PMK.06/2010,Penilai Internal di Lingkungan direktorat Jend...,2010,4,a. bahwa dalam rangka pengelolaan kekayaan ne...,"['Umum', 'Syarat Pengangkatan', 'Usulan Pengan...","['Menteri', 'Direktorat Jenderal', 'Kantor Pus...",205
6,04~pmk.07~2011.json,PMK 4/PMK.07/2011,Tata Cara Penyampaian Informasi Keuangan Daerah,2011,4,a. bahwa untuk melaksanakan Peraturan Pemerin...,[],"['Pemerintah Daerah', 'Daerah otonom,', 'Daera...",244


In [69]:
# apply api_call function to each row in docs_df
docs_df["generated_topics"] = docs_df.apply(lambda x: api_call(model, generation_prompt.format(doc_title=x["judul"], doc_consideration=x["konsideran"]), 500, 0.5, 0), axis=1)

In [70]:
docs_df

,nama_file,nomor_peraturan,judul,tahun,nomor,konsideran,heading,definition_term,token_count,generated_topics
0,01~pmk.010~2011.json,PMK 01/PMK.010/2011,Perubahan atas Peraturan Menteri Keuangan Nomo...,2011,1,a. bahwa dalam rangka memperoleh perhitungan ...,[],NaN,258,Format laporan; Tarif premi; Cadangan premi; P...
1,02~pmk.05~2011.json,PMK 02/PMK.05/2011,Pedoman Akuntansi Dan Pelaporan Aset Berupa Ba...,2011,2,a. bahwa berdasarkan Peraturan Menteri Keuang...,"['Pengertian', 'Ruang Lingkup', 'Sistem Akunta...","['Kontraktor Kontrak Kerja Sama,', 'Barang yan...",397,Pedoman Akuntansi; Pelaporan Aset; Barang Mili...
4,04~pmk.02~2014.json,PMK 04/PMK.02/2014,Perubahan Kedua atas Peraturan Menteri Keuanga...,2014,4,a. bahwa dalam rangka mendukung agar kondisi ...,[],NaN,341,Perimbangan kekayaan; Kewajiban badan penyelen...
5,04~pmk.06~2010.json,PMK 04/PMK.06/2010,Penilai Internal di Lingkungan direktorat Jend...,2010,4,a. bahwa dalam rangka pengelolaan kekayaan ne...,"['Umum', 'Syarat Pengangkatan', 'Usulan Pengan...","['Menteri', 'Direktorat Jenderal', 'Kantor Pus...",205,Penilaian Internal; Pengelolaan Kekayaan Negar...
6,04~pmk.07~2011.json,PMK 4/PMK.07/2011,Tata Cara Penyampaian Informasi Keuangan Daerah,2011,4,a. bahwa untuk melaksanakan Peraturan Pemerin...,[],"['Pemerintah Daerah', 'Daerah otonom,', 'Daera...",244,Sistem Informasi Keuangan; Peraturan Pemerinta...
